In [4]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf, linewidth=140)
import cv2
import matplotlib.pyplot as plt

In [9]:
class Image:

    def __init__(self) -> None:
        pass   
            
    def align(self,refx,refy,x1,y1,img):
        rows, cols = img.shape
        if (refx==x1) and (refy==y1):
            return img
        else:
            difx = refx - x1
            dify = refy - y1
            if (dify>0):
                crop_img = img[0:rows,(cols-dify):cols]
                img = img[0:rows,0:(cols-dify)]
                im_h = cv2.hconcat([crop_img, img])
                img = im_h

            elif (dify<0):
                dify = abs(dify)
                crop_img = img[0:rows,0:dify]
                img = img[0:rows,dify:cols]
                im_h = cv2.hconcat([img,crop_img])
                img = im_h
            else:
                img = img

            if (difx>0):
                crop_img = img[0:difx,0:cols]
                img = img[difx:rows,0:cols]
                im_v = cv2.vconcat([img,crop_img])
                img = im_v

            elif (difx<0):
                difx = abs(difx)
                crop_img = img[(rows-difx):rows,0:cols]
                img = img[0:(rows-difx),0:cols]
                im_v = cv2.vconcat([crop_img, img])
                img = im_v
            else:
                img = img
                #plt.imshow(im_v)
            return img

def main(path,excel_path_1,excel_path_2,l_thresh,r_thresh):

    obj = Image()

    df_1 = pd.read_csv(excel_path_1)
    df_2 = pd.read_csv(excel_path_2)
    refx, refy = df_1['midx'].iloc[0], df_1['midy'].iloc[0]


    edge_img_csv1 = []
    for i in range(len(df_1.index)):
        img_name = df_1['File name'].iloc[i]
        img = f'{path}/{img_name}'
        img_x, img_y = df_1['midx'].iloc[i], df_1['midy'].iloc[i]
        img = cv2.imread(img,0)
        align_img = obj.align(refx,refy,img_x,img_y,img)
        edge_detection = cv2.Canny(align_img,l_thresh,r_thresh)
        edge_img_csv1.append(edge_detection)

    # print(len(edge_img_csv1))

    hei,wid = edge_img_csv1[0].shape
    master_img = np.zeros([hei, wid])


    N = len(edge_img_csv1)
    for im in edge_img_csv1:
        master_img=master_img + im 

    master_img[master_img >= 255]=255

    csv2_error_edge = []
    for i in range(len(df_2.index)):
        img_name = df_2['File name'].iloc[i]
        img_2 = f'{path}/{img_name}'
        img_x, img_y = df_2['midx'].iloc[i], df_2['midy'].iloc[i]
        img_2 = cv2.imread(img_2,0)
        align_img = obj.align(refx,refy,img_x,img_y,img_2)
        edge_detection = cv2.Canny(align_img,l_thresh,r_thresh)
        error_edge = edge_detection - master_img
        error_edge[error_edge < 255] = 0
    
        cv2.imwrite(f'{path}/out_{img_name}',error_edge)

path = '/home/office5/Image_Code/Piyush_Sir/Images_p/'
excel_path_1 = '/home/office5/Image_Code/Piyush_Sir/points_1.csv'
excel_path_2 = '/home/office5/Image_Code/Piyush_Sir/points_2.csv'
l_thresh = 100
r_thresh = 200
a = main(path,excel_path_1,excel_path_2,l_thresh,r_thresh)

Herer
